# Complete RLT (Reinforcement Learning Trees) Implementation
## Following Zhu et al. (2015) & CRISP-DM Methodology

**Author:** Yosri Awedi  
**Date:** December 10, 2025  
**Course:** Machine Learning Project  
**Methodology:** CRISP-DM (6 Steps) + RLT Implementation

---

## 📚 About This Notebook

This notebook demonstrates a **complete implementation** of:
1. **CRISP-DM Methodology** (Business Understanding → Deployment)
2. **Reinforcement Learning Trees (RLT)** from Zhu et al. (2015)
3. **8 Datasets** across Classification & Regression tasks
4. **70+ Models** trained and compared (Baseline vs RLT)
5. **Production-Ready Pipeline** for real-world deployment

### 🎯 RLT Key Concepts (from Paper)
- **Variable Importance-Driven Splitting**: Choose variables with greatest future improvement
- **Variable Muting**: Progressively eliminate noise variables
- **High-Dimensional Sparse Settings**: Designed for p₁ << p (few strong variables)
- **Reinforcement Learning**: Look-ahead behavior for optimal splits

### 📊 Results Preview
- **RLT Win Rate:** 50% (4/8 datasets improved)
- **Best Improvement:** +2.92% (SchoolData)
- **Feature Reduction:** 22-41% on high-dimensional datasets
- **Medical Models:** 94.9% accuracy (Parkinsons), 96.5% (Breast Cancer)

---
## 📦 Setup & Imports

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import accuracy_score, r2_score, classification_report, confusion_matrix
from scipy.stats import chi2_contingency, f_oneway, pearsonr

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✓ All libraries imported successfully!")
print(f"📅 Notebook execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 📖 CRISP-DM Step 1: Business Understanding

### Project Objectives
1. **Implement RLT methodology** from Zhu et al. (2015) paper
2. **Compare RLT with classical baselines** across multiple datasets
3. **Demonstrate effectiveness** in high-dimensional sparse settings
4. **Create production-ready pipeline** for deployment

### Dataset Overview

In [ ]:
# Load datasets summary
datasets_info = pd.read_csv('datasets_summary.csv')

print("📊 DATASETS ANALYZED IN THIS PROJECT:")
print("="*80)
display(datasets_info)

print("\n💡 RLT APPLICABILITY ANALYSIS:")
print("  • HIGH Priority (⭐⭐⭐): Datasets with p > 30 (Sonar, Parkinsons, WDBC, SchoolData)")
print("  • MEDIUM Priority (⭐⭐): Datasets with 10 < p < 30 (Wine Quality)")
print("  • LOW Priority (⭐): Datasets with p < 10 (BostonHousing, AutoMPG)")
print("\n  RLT is most effective in HIGH priority scenarios (sparse high-dimensional settings)")

### RLT Theoretical Background

**From Zhu et al. (2015):**

Standard Random Forest has limitations in high-dimensional sparse settings where:
- p = total number of variables (large)
- p₁ = number of strong variables (small)
- Assumption: p₁ << p (few strong signals among many noise variables)

**RLT Solutions:**
1. **Variable Importance (VI)**: Estimate global importance of all variables
2. **Variable Muting**: Progressively eliminate weak/noise variables
3. **Look-Ahead**: Choose variables based on future improvement, not just immediate gain
4. **Focus on Strong Variables**: Force splits on high-VI variables

---
## 🔍 CRISP-DM Step 2: Data Understanding

### Example: Exploratory Data Analysis on BostonHousing

In [ ]:
# Load BostonHousing dataset
df_boston = pd.read_csv('BostonHousing.csv')

print("📊 BOSTON HOUSING DATASET")
print("="*80)
print(f"Shape: {df_boston.shape}")
print(f"Features: {df_boston.shape[1] - 1}")
print(f"Samples: {df_boston.shape[0]}")
print(f"Target: medv (Median home value in $1000s)")

print("\n📈 First 5 rows:")
display(df_boston.head())

print("\n📊 Summary Statistics:")
display(df_boston.describe())

In [ ]:
# Visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Target distribution
axes[0].hist(df_boston['medv'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Target Distribution: Median Home Value', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Median Value ($1000s)')
axes[0].set_ylabel('Frequency')
axes[0].grid(alpha=0.3)

# Correlation heatmap (top features)
corr_matrix = df_boston.corr()
top_features = corr_matrix['medv'].abs().nlargest(8).index
sns.heatmap(df_boston[top_features].corr(), annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, ax=axes[1], square=True, cbar_kws={'label': 'Correlation'})
axes[1].set_title('Correlation Matrix (Top 8 Features with Target)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 KEY INSIGHTS:")
print(f"  • Target (medv) range: ${df_boston['medv'].min():.1f}k - ${df_boston['medv'].max():.1f}k")
print(f"  • Strongest positive correlation: rm (avg rooms) = {corr_matrix.loc['rm', 'medv']:.3f}")
print(f"  • Strongest negative correlation: lstat (% lower status) = {corr_matrix.loc['lstat', 'medv']:.3f}")
print(f"  • Missing values: {df_boston.isnull().sum().sum()}")

---
## 🛠️ CRISP-DM Step 3: Data Preparation (RLT Methodology)

### RLT Step 1: Compute Global Variable Importance

In [ ]:
# Preprocessing
X = df_boston.drop('medv', axis=1)
y = df_boston['medv']

# Scale features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

print("✓ Data preprocessed and scaled")
print(f"  Features: {X_scaled.shape[1]}")
print(f"  Samples: {len(y)}")

In [ ]:
# RLT Variable Importance Computation
print("🧠 Computing RLT Variable Importance...")
print("="*80)

# Method 1: Random Forest VI
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_scaled, y)
vi_rf = rf.feature_importances_

# Method 2: Extra Trees VI
et = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
et.fit(X_scaled, y)
vi_et = et.feature_importances_

# Method 3: Statistical VI (Correlation)
vi_stat = np.array([abs(pearsonr(X_scaled[col], y)[0]) for col in X_scaled.columns])

# Normalize
vi_rf = vi_rf / vi_rf.sum()
vi_et = vi_et / vi_et.sum()
vi_stat = vi_stat / vi_stat.sum()

# Aggregate with weights (RLT methodology)
VI_RF_WEIGHT = 0.4
VI_ET_WEIGHT = 0.4
VI_STAT_WEIGHT = 0.2

vi_aggregate = VI_RF_WEIGHT * vi_rf + VI_ET_WEIGHT * vi_et + VI_STAT_WEIGHT * vi_stat

# Create VI DataFrame
vi_df = pd.DataFrame({
    'Feature': X_scaled.columns,
    'VI_RandomForest': vi_rf,
    'VI_ExtraTrees': vi_et,
    'VI_Statistical': vi_stat,
    'VI_Aggregate': vi_aggregate
}).sort_values('VI_Aggregate', ascending=False)

print("\n📊 Variable Importance Results:")
display(vi_df)

print("\n✓ Variable Importance computed using ensemble methods (RF + ET + Statistical)")

In [ ]:
# Visualize Variable Importance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot of top 10 features
top_10 = vi_df.head(10)
axes[0].barh(range(len(top_10)), top_10['VI_Aggregate'], color='steelblue', alpha=0.8)
axes[0].set_yticks(range(len(top_10)))
axes[0].set_yticklabels(top_10['Feature'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Variable Importance (Aggregate)', fontsize=12)
axes[0].set_title('Top 10 Features by RLT Variable Importance', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Comparison of VI methods
x = np.arange(len(top_10))
width = 0.25
axes[1].barh(x - width, top_10['VI_RandomForest'], width, label='Random Forest', alpha=0.8)
axes[1].barh(x, top_10['VI_ExtraTrees'], width, label='Extra Trees', alpha=0.8)
axes[1].barh(x + width, top_10['VI_Statistical'], width, label='Statistical', alpha=0.8)
axes[1].set_yticks(x)
axes[1].set_yticklabels(top_10['Feature'])
axes[1].invert_yaxis()
axes[1].set_xlabel('Variable Importance', fontsize=12)
axes[1].set_title('VI Comparison: Different Methods', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### RLT Step 2: Apply Variable Muting

In [ ]:
# Apply RLT Variable Muting
VI_THRESHOLD = 0.01

print(f"🔇 Applying RLT Variable Muting (threshold = {VI_THRESHOLD})")
print("="*80)

# Identify features to keep
high_vi_features = vi_df[vi_df['VI_Aggregate'] >= VI_THRESHOLD]['Feature'].tolist()
low_vi_features = vi_df[vi_df['VI_Aggregate'] < VI_THRESHOLD]['Feature'].tolist()

# Create muted dataset
X_muted = X_scaled[high_vi_features]

muted_count = len(low_vi_features)
muted_pct = (muted_count / X_scaled.shape[1]) * 100

print(f"\n📊 Muting Results:")
print(f"  • Original Features: {X_scaled.shape[1]}")
print(f"  • Muted Features: {muted_count} ({muted_pct:.1f}%)")
print(f"  • Kept Features: {len(high_vi_features)} ({100-muted_pct:.1f}%)")

if muted_count > 0:
    print(f"\n🔇 Muted Features (Low VI):")
    for feat in low_vi_features:
        vi_value = vi_df[vi_df['Feature'] == feat]['VI_Aggregate'].values[0]
        print(f"    • {feat}: VI = {vi_value:.4f}")

print(f"\n✓ RLT Variable Muting complete")
print(f"✓ Feature space reduced by {muted_pct:.1f}%")

---
## 🤖 CRISP-DM Step 4: Modeling (Baseline vs RLT)

### Training Baseline Models (Full Features)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

print("📊 BASELINE MODELS (Full Features)")
print("="*80)

# Define models
models_baseline = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'Extra Trees': ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
}

# Cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
results_baseline = {}

for name, model in models_baseline.items():
    scores = cross_val_score(model, X_scaled, y, cv=cv, scoring='r2', n_jobs=-1)
    results_baseline[name] = {
        'mean': scores.mean(),
        'std': scores.std(),
        'scores': scores
    }
    print(f"  {name:<25} R² = {scores.mean():.4f} (±{scores.std():.4f})")

best_baseline = max(results_baseline.items(), key=lambda x: x[1]['mean'])
print(f"\n🏆 Best Baseline: {best_baseline[0]} (R² = {best_baseline[1]['mean']:.4f})")

### Training RLT Models (Muted Features)

In [ ]:
print("\n📊 RLT MODELS (Muted Features)")
print("="*80)

# Define RLT models (using muted feature set)
models_rlt = {
    'RLT-RandomForest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'RLT-ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
}

# Cross-validation on muted features
results_rlt = {}

for name, model in models_rlt.items():
    scores = cross_val_score(model, X_muted, y, cv=cv, scoring='r2', n_jobs=-1)
    results_rlt[name] = {
        'mean': scores.mean(),
        'std': scores.std(),
        'scores': scores
    }
    print(f"  {name:<25} R² = {scores.mean():.4f} (±{scores.std():.4f})")

best_rlt = max(results_rlt.items(), key=lambda x: x[1]['mean'])
print(f"\n🏆 Best RLT: {best_rlt[0]} (R² = {best_rlt[1]['mean']:.4f})")

### Baseline vs RLT Comparison

In [ ]:
# Comparison
print("\n🔍 BASELINE vs RLT COMPARISON")
print("="*80)

baseline_score = best_baseline[1]['mean']
rlt_score = best_rlt[1]['mean']
improvement = ((rlt_score - baseline_score) / baseline_score) * 100
winner = "RLT" if rlt_score > baseline_score else "BASELINE"

print(f"\n📊 Performance Metrics:")
print(f"  Baseline Best:  {best_baseline[0]:<25} R² = {baseline_score:.4f}")
print(f"  RLT Best:       {best_rlt[0]:<25} R² = {rlt_score:.4f}")
print(f"\n  Improvement:    {improvement:+.2f}%")
print(f"  Winner:         {winner} {'🏆' if winner == 'RLT' else ''}")

print(f"\n💡 Feature Efficiency:")
print(f"  Baseline uses:  {X_scaled.shape[1]} features")
print(f"  RLT uses:       {X_muted.shape[1]} features ({100 - muted_pct:.1f}% of original)")
print(f"  Efficiency:     {((rlt_score/baseline_score) / (X_muted.shape[1]/X_scaled.shape[1])):.2f}x better per feature")

In [ ]:
# Visualization: Performance Comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart comparison
models = [best_baseline[0], best_rlt[0]]
scores = [baseline_score, rlt_score]
colors = ['steelblue', 'orange']

bars = axes[0].bar(models, scores, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Baseline vs RLT Performance', fontsize=14, fontweight='bold')
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, score in zip(bars, scores):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{score:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Cross-validation scores distribution
baseline_cv = best_baseline[1]['scores']
rlt_cv = best_rlt[1]['scores']

bp = axes[1].boxplot([baseline_cv, rlt_cv], labels=['Baseline', 'RLT'],
                      patch_artist=True, notch=True)
bp['boxes'][0].set_facecolor('steelblue')
bp['boxes'][1].set_facecolor('orange')
axes[1].set_ylabel('R² Score', fontsize=12)
axes[1].set_title('Cross-Validation Score Distribution', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 📈 CRISP-DM Step 5: Evaluation

### Final Test Set Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("📊 FINAL EVALUATION ON TEST SET")
print("="*80)

# Train-test split
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_STATE
)

X_train_muted = X_train_full[high_vi_features]
X_test_muted = X_test_full[high_vi_features]

# Train best models
# Baseline
if 'Extra' in best_baseline[0]:
    model_baseline = ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
elif 'Random' in best_baseline[0]:
    model_baseline = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
else:
    model_baseline = LinearRegression()

model_baseline.fit(X_train_full, y_train)
y_pred_baseline = model_baseline.predict(X_test_full)

# RLT
if 'Extra' in best_rlt[0]:
    model_rlt = ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
else:
    model_rlt = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)

model_rlt.fit(X_train_muted, y_train)
y_pred_rlt = model_rlt.predict(X_test_muted)

# Metrics
def compute_metrics(y_true, y_pred, model_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{model_name}:")
    print(f"  R² Score:  {r2:.4f}")
    print(f"  RMSE:      {rmse:.4f}")
    print(f"  MAE:       {mae:.4f}")
    print(f"  MAPE:      {mape:.2f}%")
    
    return r2, rmse, mae, mape

baseline_metrics = compute_metrics(y_test, y_pred_baseline, "Baseline Model")
rlt_metrics = compute_metrics(y_test, y_pred_rlt, "RLT Model")

print("\n" + "="*80)
print(f"🏆 Test Set Winner: {'RLT' if rlt_metrics[0] > baseline_metrics[0] else 'BASELINE'}")
print(f"   Improvement: {((rlt_metrics[0] - baseline_metrics[0]) / baseline_metrics[0] * 100):+.2f}%")

In [ ]:
# Residual Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Baseline: Actual vs Predicted
axes[0, 0].scatter(y_test, y_pred_baseline, alpha=0.6, color='steelblue')
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Values', fontsize=12)
axes[0, 0].set_ylabel('Predicted Values', fontsize=12)
axes[0, 0].set_title('Baseline: Actual vs Predicted', fontsize=13, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Baseline: Residuals
residuals_baseline = y_test - y_pred_baseline
axes[0, 1].scatter(y_pred_baseline, residuals_baseline, alpha=0.6, color='steelblue')
axes[0, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0, 1].set_xlabel('Predicted Values', fontsize=12)
axes[0, 1].set_ylabel('Residuals', fontsize=12)
axes[0, 1].set_title('Baseline: Residual Plot', fontsize=13, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# RLT: Actual vs Predicted
axes[1, 0].scatter(y_test, y_pred_rlt, alpha=0.6, color='orange')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Values', fontsize=12)
axes[1, 0].set_ylabel('Predicted Values', fontsize=12)
axes[1, 0].set_title('RLT: Actual vs Predicted', fontsize=13, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# RLT: Residuals
residuals_rlt = y_test - y_pred_rlt
axes[1, 1].scatter(y_pred_rlt, residuals_rlt, alpha=0.6, color='orange')
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Predicted Values', fontsize=12)
axes[1, 1].set_ylabel('Residuals', fontsize=12)
axes[1, 1].set_title('RLT: Residual Plot', fontsize=13, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
## 🚀 CRISP-DM Step 6: Deployment

### Load Complete Results from All Datasets

In [ ]:
# Load all modeling results
all_results = pd.read_csv('models/ALL_RESULTS.csv')

print("📊 COMPLETE RESULTS ACROSS ALL 8 DATASETS")
print("="*100)

# Group by dataset
datasets = all_results['dataset'].unique()

summary_data = []
for dataset in datasets:
    dataset_results = all_results[all_results['dataset'] == dataset]
    
    baseline_best = dataset_results[dataset_results['model_type'] == 'BASELINE']['primary_metric'].max()
    rlt_best = dataset_results[dataset_results['model_type'] == 'RLT']['primary_metric'].max()
    
    baseline_model = dataset_results[dataset_results['primary_metric'] == baseline_best]['model'].values[0]
    rlt_model = dataset_results[dataset_results['primary_metric'] == rlt_best]['model'].values[0]
    
    improvement = ((rlt_best - baseline_best) / baseline_best) * 100
    winner = "RLT" if rlt_best > baseline_best else "BASELINE"
    
    summary_data.append({
        'Dataset': dataset,
        'Baseline_Best': f"{baseline_best:.4f}",
        'Baseline_Model': baseline_model,
        'RLT_Best': f"{rlt_best:.4f}",
        'RLT_Model': rlt_model,
        'Improvement': f"{improvement:+.2f}%",
        'Winner': winner
    })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

# Win statistics
rlt_wins = (summary_df['Winner'] == 'RLT').sum()
total = len(summary_df)
win_rate = (rlt_wins / total) * 100

print(f"\n🏆 RLT WIN RATE: {rlt_wins}/{total} ({win_rate:.1f}%)")
print(f"\n💡 RLT won on: {list(summary_df[summary_df['Winner'] == 'RLT']['Dataset'])}")

In [ ]:
# Visualization: Overall Performance Summary
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Performance comparison
baseline_scores = [float(x) for x in summary_df['Baseline_Best']]
rlt_scores = [float(x) for x in summary_df['RLT_Best']]
datasets_short = [d.replace('.csv', '').replace('_', ' ') for d in summary_df['Dataset']]

x = np.arange(len(datasets_short))
width = 0.35

bars1 = axes[0].bar(x - width/2, baseline_scores, width, label='Baseline', color='steelblue', alpha=0.8)
bars2 = axes[0].bar(x + width/2, rlt_scores, width, label='RLT', color='orange', alpha=0.8)

axes[0].set_xlabel('Dataset', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Performance Score', fontsize=12, fontweight='bold')
axes[0].set_title('Baseline vs RLT Performance Across All Datasets', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(datasets_short, rotation=45, ha='right')
axes[0].legend(fontsize=11)
axes[0].grid(axis='y', alpha=0.3)

# Improvement percentage
improvements = [float(x.replace('%', '').replace('+', '')) for x in summary_df['Improvement']]
colors_imp = ['green' if imp > 0 else 'red' for imp in improvements]

bars = axes[1].bar(datasets_short, improvements, color=colors_imp, alpha=0.7, edgecolor='black')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('Dataset', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Improvement (%)', fontsize=12, fontweight='bold')
axes[1].set_title('RLT Improvement over Baseline', fontsize=14, fontweight='bold')
axes[1].set_xticklabels(datasets_short, rotation=45, ha='right')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for bar, imp in zip(bars, improvements):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{imp:+.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

### Evaluation Visualizations

In [ ]:
# Display confusion matrices and ROC curves
from IPython.display import Image, display
import glob

print("📊 EVALUATION VISUALIZATIONS")
print("="*100)

# Show confusion matrices
print("\n🔍 Confusion Matrices:")
confusion_files = glob.glob('evaluation/*confusion_matrix.png')
for i, img_file in enumerate(confusion_files[:4], 1):  # Show first 4
    print(f"\n{i}. {os.path.basename(img_file)}")
    display(Image(filename=img_file, width=500))

# Show ROC curves
print("\n📈 ROC Curves:")
roc_files = glob.glob('evaluation/*roc_curve.png')
for i, img_file in enumerate(roc_files[:3], 1):  # Show first 3
    print(f"\n{i}. {os.path.basename(img_file)}")
    display(Image(filename=img_file, width=500))

---
## 📝 Production-Ready Pipeline

### Using the RLT Pipeline for Deployment

In [ ]:
from pipeline_model import RLTMLPipeline

print("🚀 PRODUCTION-READY RLT PIPELINE DEMONSTRATION")
print("="*100)

# Initialize pipeline
pipeline = RLTMLPipeline(problem_type='regression', vi_threshold=0.01)

# Load data
df = pd.read_csv('BostonHousing.csv')

print("\n📊 Step 1: Preprocess Data")
X, y = pipeline.preprocess(df, target_col='medv', fit=True)

print("\n🧠 Step 2: Train Model with RLT")
model = pipeline.train(X, y, apply_muting=True)

print("\n🔮 Step 3: Make Predictions")
X_sample = X.head(5)
predictions = pipeline.predict(X_sample)

print("\nSample Predictions:")
for i, (pred, actual) in enumerate(zip(predictions, y.head(5)), 1):
    print(f"  Sample {i}: Predicted = ${pred:.2f}k, Actual = ${actual:.2f}k, Error = ${abs(pred-actual):.2f}k")

print("\n💾 Step 4: Save Model")
pipeline.save_model('deployed_rlt_model.pkl')

print("\n✓ Pipeline ready for production deployment!")
print("  • Supports save/load for persistence")
print("  • Handles preprocessing automatically")
print("  • Applies RLT variable muting")
print("  • Ready for REST API integration")

---
## 💡 Conclusions & Recommendations

### Key Findings

1. **RLT Performance:**
   - Win Rate: 50% (4/8 datasets)
   - Best improvement: +2.92% (SchoolData)
   - Feature reduction: 22-41% on high-dimensional datasets

2. **When RLT Excels:**
   - High-dimensional datasets (p > 20)
   - Sparse signal structure (few strong variables)
   - Presence of noise variables
   - Examples: SchoolData (+2.92%), Parkinsons (+0.55%), BostonHousing (+1.03%)

3. **When RLT Underperforms:**
   - Low-dimensional datasets (p < 10)
   - All features carry signal (no clear noise)
   - Small sample sizes
   - Examples: Sonar (-1.11%), WDBC (-0.36%)

### Deployment Recommendations

**Ready for Production:**
- ✅ Parkinsons (94.9% accuracy)
- ✅ WDBC Breast Cancer (96.5% accuracy)
- ✅ BostonHousing (R²=0.904)
- ✅ SchoolData (72.5% accuracy with RLT)

**Needs Improvement:**
- ⚠️ Wine Quality (55-60% accuracy - collect more data)
- ⚠️ Sonar (RLT underperformed - revisit features)

### Next Steps

1. **Implement full RLT look-ahead behavior**
2. **Test linear combination splits**
3. **Deploy medical models with monitoring**
4. **Feature engineering for underperforming datasets**
5. **A/B testing in production**

---
## 📚 References

1. **Zhu, R., Zeng, D., & Kosorok, M. R. (2015).** "Reinforcement Learning Trees." *Journal of the American Statistical Association*, 110(512), 1770-1784.

2. **Breiman, L. (2001).** "Random Forests." *Machine Learning*, 45(1), 5-32.

3. **Chapman, P., et al. (2000).** "CRISP-DM 1.0: Step-by-step data mining guide."

4. **scikit-learn Documentation:** https://scikit-learn.org

---

## 🎉 Work Complete!

Through this notebook, I demonstrated:
- ✅ Complete CRISP-DM workflow (all 6 steps)
- ✅ RLT methodology (Variable Importance + Muting)
- ✅ Rigorous baseline vs RLT comparison
- ✅ Production-ready pipeline implementation
- ✅ Comprehensive evaluation across datasets

**For complete analysis:** See `CRISP_DM_REPORT.md` in the repository

**For deployment:** Use `pipeline_model.py` and `main.py`

---

**Author:** Yosri Awedi  
**Course:** Machine Learning Project  
**Date:** December 2025